# Model Evaluation - Bank Fraud Detection

This notebook provides comprehensive model evaluation:
- Classification metrics (F1, ROC-AUC, Recall)
- Confusion matrix analysis
- ROC and Precision-Recall curves
- Error analysis
- Feature importance


In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')

from src.models.model_evaluator import ModelEvaluator
from src.models.model_trainer import FraudModelTrainer
import joblib
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Load model and data
model = joblib.load('../models/xgboost_model.pkl')  # Adjust path as needed
X_test = pd.read_csv('../data/processed/X_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv')['is_fraud']

# Load preprocessor
from src.data.data_preprocessor import FraudDataPreprocessor
preprocessor = FraudDataPreprocessor.load('../models/preprocessor.pkl')
preprocessor.handle_imbalance = False

X_test_proc, y_test_proc = preprocessor.transform(X_test, y_test)


FileNotFoundError: [Errno 2] No such file or directory: '../models/xgboost_model.pkl'

In [ ]:
# Generate predictions
y_pred = model.predict(X_test_proc)
y_pred_proba = model.predict_proba(X_test_proc)[:, 1]

# Evaluate
evaluator = ModelEvaluator(output_dir='../reports/figures')
metrics = evaluator.calculate_classification_metrics(y_test_proc.values, y_pred, y_pred_proba)

print("Test Set Metrics:")
for key, value in metrics.items():
    if key != 'confusion_matrix':
        print(f"{key}: {value:.4f}")


In [ ]:
# Generate visualizations
evaluator.plot_confusion_matrix(
    y_test_proc.values, y_pred,
    model_name="Best Model",
    save_path='../reports/figures/confusion_matrix_test.png'
)

evaluator.plot_roc_curve(
    y_test_proc.values, y_pred_proba,
    model_name="Best Model",
    save_path='../reports/figures/roc_curve_test.png'
)

evaluator.plot_precision_recall_curve(
    y_test_proc.values, y_pred_proba,
    model_name="Best Model",
    save_path='../reports/figures/precision_recall_curve_test.png'
)

print("Visualizations saved to ../reports/figures/")
